# Tomography Debugger
State tomography involves measuring a quantum state repeatedly in the bases given by `itertools.product(['X', 'Y', 'Z], repeat=n_qubits)`. From these measurements, we can reconstruct a density matrix $\rho$ using a varaiety of methods described in forest.benchmarking.tomography under the heading "state tomography". This is all done automaticly in using the forest.benchmarking.tomography.tomographize function allowing it to be use effectivly as a quantum debugger.

In [1]:
import numpy as np
import time

from pyquil import Program, get_qc
from pyquil.gates import *
from forest.benchmarking.tomography import tomographize

## Construct a state with a `Program`
We'll construct a two-qubit graph state by Hadamarding all qubits and then applying a controlled-Z operation across edges of our graph. In the two-qubit case, there's only one edge. 

In [2]:
qubits = [0, 1]

program = Program()
for qubit in qubits:
    program += H(qubit)
program += CZ(qubits[0], qubits[1])
program += RY(-np.pi/2, qubits[0])
program += X(qubits[1])
program += CNOT(qubits[1], qubits[0])

print(program)

H 0
H 1
CZ 0 1
RY(-pi/2) 0
X 1
CNOT 1 0



## Run the tomography debugger and print output

In [3]:
qc = get_qc('%dq-qvm' % len(qubits))


start_linear = time.time()
m = 10
rho_linear = tomographize(qc, program, qubits, pauli_num=10, t_type="linear_inv")
end_linear = time.time() - start_linear

print("Linear tomography took %gs" % end_linear)
print("Recovered density matrix:\n")
print(rho_linear)

start_compressed = time.time()
rho_compressed = tomographize(qc, program, qubits, pauli_num=10, t_type="compressed_sensing")
end_compressed = time.time() - start_compressed
print("Compressed tomography took %gs" % end_compressed)
print("Recovered density matrix:\n")
print(rho_compressed)

start_lasso = time.time()
rho_lasso = tomographize(qc, program, qubits, pauli_num=10, t_type="lasso")
end_lasso = time.time() - start_lasso
print("Compressed tomography took %gs" % end_lasso)
print("Recovered density matrix:\n")
print(rho_lasso)

Linear tomography took 1.5077s
Recovered density matrix:

[[ 0.249-8.57061622e-18j  0.25 -7.50000000e-03j  0.003+1.30000000e-02j
   0.012-5.00000000e-03j]
 [ 0.25 +7.50000000e-03j  0.251-1.42843604e-17j  0.005-5.00000000e-03j
  -0.017-3.00000000e-03j]
 [ 0.003-1.30000000e-02j  0.005+5.00000000e-03j  0.249-2.85687207e-18j
  -0.25 +7.50000000e-03j]
 [ 0.012+5.00000000e-03j -0.017+3.00000000e-03j -0.25 -7.50000000e-03j
   0.251+2.85687207e-18j]]


NameError: name 'm' is not defined

## Compare results to true output obtained using wavefunction simulator

In [ ]:
from pyquil.api import WavefunctionSimulator
wf_sim = WavefunctionSimulator()
wf = wf_sim.wavefunction(program)
psi = wf.amplitudes

rho_true = np.outer(psi, psi.T.conj())
print(np.around(rho_true, decimals=3))

## Visualize using Hinton plots

In [ ]:
from matplotlib import pyplot as plt
from forest.benchmarking.plotting import hinton
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
hinton(rho_true, ax=ax1)
hinton(rho_linear, ax=ax2)
hinton(rho_compressed, ax=ax3)
ax1.set_title('Analytical Linear')
ax2.set_title('Estimated Linear')
ax3.set_title('Estimated Compressed')
fig.tight_layout()

## Calculate matrix norm between true and estimated rho

In [ ]:
print("Linear norm:")
print(np.linalg.norm(rho_linear - rho_true))
print("Compressed norm:")
print(np.linalg.norm(rho_compressed - rho_true))
print("Lasso norm:")
print(np.linalg.norm(rho_lasso - rho_true))

## Plot graph of results for various measurement values

In [ ]:
import matplotlib.pyplot as plt

max_pauli_num = 4 ** len(qubits)
num_trials = 5

linear_norms = []
compressed_norms = []

print("Analyzing performance of linear vs. compressed on program:")
print(program)

for i in range(1, max_pauli_num):
    print("Running iteration %d/%d" % (i, max_pauli_num))
    linear_norm_mean = 0.0
    compressed_norm_mean = 0.0
    for j in range(num_trials):
        rho_linear = tomographize(qc, program, qubits, pauli_num=i, t_type="linear_inv")
        rho_compressed = tomographize(qc, program, qubits, pauli_num=i, t_type="compressed_sensing")
        linear_norm_mean += np.linalg.norm(rho_linear - rho_true)
        compressed_norm_mean += np.linalg.norm(rho_compressed - rho_true)
    
    linear_norm_mean /= num_trials
    compressed_norm_mean /= num_trials
    
    linear_norms.append(linear_norm_mean)
    compressed_norms.append(compressed_norm_mean)

plt.plot(linear_norms, label='linear')
plt.plot(compressed_norms, label='compressed')
plt.legend()
plt.show()